In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.rcParams["figure.figsize"] = (15, 5)
from pmdarima import auto_arima, model_selection, ARIMA
from pmdarima.arima.utils import ndiffs
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from datetime import datetime, timedelta, date
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import joblib

In [6]:
countries = 'Spain'
datecol = 'date'
exog=pd.read_csv("data/exogenous.csv", parse_dates=[datecol], index_col=[datecol])
endog=pd.read_csv("data/endogenous.csv", parse_dates=[datecol], index_col=[datecol])
cases="cases"

for cases in ["cases","deaths"]:

    for country in countries.split('|'):
        #filename = './models/' + country + 'SARIMAXmodel.pkl'
        filename = './models/' + country + '_new_'+cases+'_model.pkl'
        model = joblib.load(filename)
        if model.fittedvalues.index[-1]== exog.index[-1]:
            print("up to date")
        else:

            exogmodel=exog.loc[:, exog.columns.str.contains(country)]
            endogmodel=endog.loc[:, endog.columns.str.contains("new_{}_{}".format(cases,country))]
            #exogmodel=exogmodel.iloc[-1:,:]
            sc_in = MinMaxScaler(feature_range=(0, 1))
            scaled_input = sc_in.fit_transform(exogmodel)
            scaled_input = pd.DataFrame(scaled_input, index=exogmodel.index, columns=exogmodel.columns)
            X_update = scaled_input
            X_update=X_update.iloc[-1:,:]

            sc_out = MinMaxScaler(feature_range=(0, 1))
            scaled_output = sc_out.fit_transform(endogmodel)
            scaled_output  = pd.Series(scaled_output.flatten(), index=endogmodel.index, name=endogmodel.columns[0])
            y = scaled_output.resample('1D').sum()
            y=y.iloc[-1:]
            modelupdated=model.append(y, exog=X_update, refit=True,)
            joblib.dump(modelupdated, filename)
            print("{} new_{} updated".format(country,cases))

up to date
up to date


In [12]:
filename = './models/' + "Denmark" + 'SARIMAXmodel.pkl'
model = joblib.load(filename)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:      new_cases_Denmark   No. Observations:                  260
Model:               SARIMAX(1, 1, 0)   Log Likelihood                 627.782
Date:                Thu, 17 Sep 2020   AIC                          -1223.564
Time:                        00:04:56   BIC                          -1166.655
Sample:                    01-01-2020   HQIC                         -1200.683
                         - 09-16-2020                                         
Covariance Type:                  opg                                         
================================================================================================================
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
positive_rate_Denmark                            0.4168      0.207      2.013      0.044       0.011       0.823
C1_School closing_Denmark                        0.1682      0.191      0.879      0.379      -0.207       0.543
C2_Workplace closing_Denmark                     0.3148      0.361      0.873      0.383      -0.392       1.022
C3_Cancel public events_Denmark                  0.0019      0.376      0.005      0.996      -0.734       0.738
C4_Restrictions on gatherings_Denmark            0.0453      0.347      0.130      0.896      -0.636       0.726
C5_Close public transport_Denmark                0.1424      0.445      0.320      0.749      -0.730       1.015
C6_Stay at home requirements_Denmark             0.1248      0.832      0.150      0.881      -1.505       1.755
C7_Restrictions on internal movement_Denmark    -0.5004      0.419     -1.194      0.233      -1.322       0.321
C8_International travel controls_Denmark        -0.3022      0.480     -0.629      0.529      -1.244       0.639
H1_Public information campaigns_Denmark         -0.0865      1.768     -0.049      0.961      -3.552       3.379
H2_Testing policy_Denmark                       -0.0031      0.280     -0.011      0.991      -0.553       0.547
H3_Contact tracing_Denmark                       0.1362      0.193      0.706      0.480      -0.242       0.514
mask_Denmark                                     0.6233      0.141      4.421      0.000       0.347       0.900
flightsArr_Denmark                               0.0116      0.288      0.040      0.968      -0.553       0.576
ar.L1                                            0.5179      0.039     13.376      0.000       0.442       0.594
sigma2                                           0.0005    1.7e-05     26.572      0.000       0.000       0.000
===================================================================================
Ljung-Box (Q):                       82.45   Jarque-Bera (JB):              1841.09
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):              13.62   Skew:                            -0.08
Prob(H) (two-sided):                  0.00   Kurtosis:                        16.06
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [13]:
filename = './models/' + "Spain" + 'SARIMAXmodel.pkl'
model = joblib.load(filename)

In [14]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:        new_cases_Spain   No. Observations:                  260
Model:               SARIMAX(1, 1, 3)   Log Likelihood                 728.540
Date:                Thu, 17 Sep 2020   AIC                          -1419.080
Time:                        00:05:06   BIC                          -1351.501
Sample:                    01-01-2020   HQIC                         -1391.909
                         - 09-16-2020                                         
Covariance Type:                  opg                                         
==============================================================================================================
                                                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
positive_rate_Spain                            0.1592      0.073      2.180      0.029       0.016       0.302
C1_School closing_Spain                       -0.0323      0.145     -0.222      0.824      -0.317       0.253
C2_Workplace closing_Spain                     0.0248      0.139      0.179      0.858      -0.247       0.297
C3_Cancel public events_Spain                  0.2537      0.287      0.884      0.377      -0.309       0.816
C4_Restrictions on gatherings_Spain           -0.1257      0.108     -1.160      0.246      -0.338       0.087
C5_Close public transport_Spain               -0.0607      0.109     -0.559      0.576      -0.274       0.152
C6_Stay at home requirements_Spain             0.0834      0.062      1.344      0.179      -0.038       0.205
C7_Restrictions on internal movement_Spain     0.0448      0.118      0.380      0.704      -0.186       0.276
C8_International travel controls_Spain        -0.3177      0.375     -0.848      0.396      -1.052       0.416
H1_Public information campaigns_Spain          0.0036      1.586      0.002      0.998      -3.105       3.112
H2_Testing policy_Spain                       -0.1138      0.066     -1.730      0.084      -0.243       0.015
H3_Contact tracing_Spain                      -0.0288      1.550     -0.019      0.985      -3.066       3.009
mask_Spain                                     0.7295      0.510      1.431      0.153      -0.270       1.729
flightsArr_Spain                              -0.0831      0.290     -0.286      0.775      -0.652       0.486
ar.L1                                          0.8871      0.039     22.679      0.000       0.810       0.964
ma.L1                                         -0.6325      0.066     -9.577      0.000      -0.762      -0.503
ma.L2                                          0.0201      0.063      0.317      0.751      -0.104       0.144
ma.L3                                          0.1178      0.053      2.230      0.026       0.014       0.221
sigma2                                         0.0002   1.13e-05     18.193      0.000       0.000       0.000
===================================================================================
Ljung-Box (Q):                       72.59   Jarque-Bera (JB):              1172.53
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):              10.29   Skew:                             1.60
Prob(H) (two-sided):                  0.00   Kurtosis:                        12.92
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""